In [ ]:
!pip3 install pymorphy2
!pip3 install razdel
!pip3 install nltk
!pip3 install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from glob import glob
from typing import List, Dict, Optional, Union, Tuple
import re
import pymorphy2
import string
import razdel
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import nltk
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import ndcg_score
nltk.download('stopwords')
string.punctuation


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
class TextPreparer:
  raw_text: str = ''
  prepared_text: str = ''
  sentences: List[str] = []

  def __init__(self):
    pass

  def read_texts(self, text_names: Optional[List[str]] = None):
    texts = []
    if not text_names:
      text_names = sorted(glob('drive/MyDrive/texts/*'))
    for tname in text_names:
      with open(tname, 'r') as r:
        texts.append(r.read())
    self.raw_text = '\n'.join(texts)

  def clean_and_split_text(self):
    self.prepared_text = re.sub(r'\[\d+\]', '', self.raw_text)
    self.prepared_text = re.sub(r'англ\.', '', self.raw_text)
    self.prepared_text = self.prepared_text.lower()
    non_terminal_signs = ''.join([sign for sign in string.punctuation if sign not in ['!', '?', '.']])
    self.prepared_text = self.prepared_text.translate(str.maketrans('', '', non_terminal_signs))
    self.prepared_text = re.sub(r'\s+', ' ', self.prepared_text)

  def prepare_sentences(self):
    morph = pymorphy2.MorphAnalyzer()
    self.raw_sentences = re.split(r'[!\?\.]', self.prepared_text)
    self.sentences = []
    for sentence in self.raw_sentences:
      words = sentence.split(' ')
      normalized_sentence = ' '.join([morph.parse(word)[0].normal_form for word in words if word not in stopwords.words('russian')])
      normalized_sentence = normalized_sentence.strip()
      if normalized_sentence:
        self.sentences.append(normalized_sentence)

In [ ]:
t = TextPreparer()
t.read_texts()
t.clean_and_split_text()
t.prepare_sentences()

In [ ]:
[73, 107, 184, 200, 221]

[73, 107, 184, 200, 221]

In [ ]:
t.sentences[:5]

['king arthur knight’s tale — компьютерный тактический ролевый игра разработать выпустить венгерский компания neocoregames',
 'мир игра основать артуриана — цикл легенда рыцарский роман связанный британский король артур рыцарь круглый стол',
 'сюжет разворачиваться сражение камланна который предать артур сэр мордред пасть рука король всё успеть нанести тот смертельный рана',
 'действие переноситься легендарный остров авалон увозить умирающий король',
 'волшебница леди озеро пытаться вернуть артур жизнь однако вместо король превращаться чудовище волшебный остров заполонять орда нежить']

In [ ]:
relevant_sentences = {}
relevant_sentences['game'] = {
    0: 1,
    46: 2,
    60: 2,
}
relevant_sentences['riot'] = {
    119: 1,
    145: 2,
    202: 1,
}
relevant_sentences['izuerit'] = {
    74: 1,
    89: 1,
    94: 2,
}

In [ ]:
t.raw_sentences[73]

' джон джерард john gerard 4 октября 1564 предположительно ньюбрин дербишир королевство англия — 27 июля 1637 рим папская область — английский иезуит долгое время занимавшийся подпольной миссионерской деятельностью на территории англии'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm

In [ ]:
count_vectorizer = CountVectorizer(max_features=777)
tfidf_vectorizer = TfidfVectorizer(max_features=777)

In [ ]:
c_vectorized_sentences = count_vectorizer.fit_transform(t.sentences).toarray()
t_vectorized_sentences = tfidf_vectorizer.fit_transform(t.sentences).toarray()

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
# model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

def get_model_embeddings(texts: List[str]) -> np.ndarray:
  def average_pool(last_hidden_states: Tensor,
                  attention_mask: Tensor) -> Tensor:
      last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
      return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

  batch_dict = tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

  outputs = model(**batch_dict)
  embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
  return embeddings.detach().tolist()

p_vectorized_sentences = []
for i in tqdm(range(0, len(t.raw_sentences), 20)):
  p_vectorized_sentences.extend(
      get_model_embeddings(
          [f'query: {sentence}' for sentence in t.raw_sentences[i: i + 20]]
      )
  )
p_vectorized_sentences = np.array(p_vectorized_sentences)

100%|██████████| 12/12 [00:54<00:00,  4.57s/it]


In [ ]:
morph = pymorphy2.MorphAnalyzer()


def prepare_query(query: str) -> str:
  query = re.sub(r'\[\d+\]', '', query)
  query = query.lower()
  non_terminal_signs = ''.join([sign for sign in string.punctuation if sign not in ['!', '?', '.']])
  query = query.translate(str.maketrans('', '', non_terminal_signs))
  query = re.sub(r'\s+', ' ', query)
  query = ' '.join([morph.parse(word)[0].normal_form for word in query.split() if word not in stopwords.words('russian')])
  return query.strip()


def get_relevance(query: str, vectorizer: Union[TfidfVectorizer, CountVectorizer], vectorized_data: np.ndarray, topk: Optional[int] = None) -> Tuple[np.ndarray, np.ndarray]:
  query = prepare_query(query)
  features = vectorizer.transform([query])[0]
  relevance = cosine_similarity(vectorized_data, features.reshape(1, -1)).reshape(-1)
  args = np.argsort(relevance)[::-1]
  relevance_values = relevance[args]
  return relevance_values, args


def get_torch_relevance(query: str) -> Tuple[np.ndarray, np.ndarray]:
  features = get_model_embeddings([f'query: {query}'])[0]
  relevance = cosine_similarity(np.array(p_vectorized_sentences)[:-1], np.array(features).reshape(1, -1)).reshape(-1)
  args = np.argsort(relevance)[::-1]
  relevance_values = relevance[args]
  return relevance_values, args


def get_relevant_sentences(t: TextPreparer, values: np.ndarray, args: np.ndarray):
  return zip(np.round(values, 3), [t.raw_sentences[arg] for arg in args])

In [ ]:
def get_relevance_table(query: str):
  relevance_tfidf, args_tfidf = get_relevance(query, tfidf_vectorizer, t_vectorized_sentences)
  relevance_counter, args_counter = get_relevance(query, count_vectorizer, c_vectorized_sentences)
  relevance_torch, args_torch = get_torch_relevance(query)
  zipped_data = zip(
      args_tfidf,
      np.round(relevance_tfidf, 3), [t.raw_sentences[arg] for arg in args_tfidf],
      args_counter,
      np.round(relevance_counter, 3), [t.raw_sentences[arg] for arg in args_counter],
      args_torch,
      np.round(relevance_torch, 3), [t.raw_sentences[arg] for arg in args_torch],
  )
  return pd.DataFrame(zipped_data, columns=['sentence_id_t', 'relevance_tfidf', 'text_tfidf', 'sentence_id_c', 'relevance_countv', 'text_countv', 'sentence_id_p', 'relevance_pytorch', 'text_pytorch'])

In [ ]:
izu_rel = get_relevance_table('Английский иезуит смог бежать из Тауэра по верёвке, натянутой надо рвом.')
izu_rel

,sentence_id_t,relevance_tfidf,text_tfidf,sentence_id_c,relevance_countv,text_countv,sentence_id_p,relevance_pytorch,text_pytorch
0,94,0.529,в октябре 1597 года он смог бежать из тауэра ...,94,0.548,в октябре 1597 года он смог бежать из тауэра ...,94,0.900,в октябре 1597 года он смог бежать из тауэра ...
1,74,0.375,был схвачен но выдержал пытки и смог бежать,74,0.400,был схвачен но выдержал пытки и смог бежать,174,0.846,существует поверье что если птицы покинут тау...
2,89,0.298,там он учился в английском колледже был рукоп...,170,0.316,английских стражей нередко называют бифитерам...,214,0.841,оставшиеся в саутварке непримиримые сторонник...
3,210,0.238,король вынужден был бежать из столицы,89,0.298,там он учился в английском колледже был рукоп...,74,0.840,был схвачен но выдержал пытки и смог бежать
4,170,0.225,английских стражей нередко называют бифитерам...,172,0.258,этим английская корона обеспечивала себе надё...,131,0.836,немало особ королевского рода представлявших ...
...,...,...,...,...,...,...,...,...,...
216,141,0.000,отсеченную голову надевали на кол и выставлял...,141,0.000,отсеченную голову надевали на кол и выставлял...,30,0.736,основная кампания игра делится на принятие ст...
217,140,0.000,большая часть других казней — в основном обез...,140,0.000,большая часть других казней — в основном обез...,8,0.735,разработка игры была начата в 2019 году
218,139,0.000,трое из этих женщин были королевами — это анн...,139,0.000,трое из этих женщин были королевами — это анн...,13,0.734,дополнительные режимы включают в себя мультип...
219,138,0.000,хотя в тауэр были брошены тысячи заключённых ...,138,0.000,хотя в тауэр были брошены тысячи заключённых ...,206,0.733,реально руководство принадлежало представител...


In [ ]:
riot_rel = get_relevance_table('Участник восстания после его разгрома попал не на эшафот, а в парламент.')
riot_rel

,sentence_id_t,relevance_tfidf,text_tfidf,sentence_id_c,relevance_countv,text_countv,sentence_id_p,relevance_pytorch,text_pytorch
0,180,0.197,символом зловещего прошлого тауэра служит мес...,207,0.200,предводителем восстания стал некий джек джон ...,192,0.845,восстание было подавлено сэр роберт оказался ...
1,193,0.184,в том же году пойнингс вышел на свободу и был...,208,0.189,программные документы восстания джэка кэда со...,98,0.828,он сделал это присоединившись в одежде слуги ...
2,185,0.162,заседал в парламенте погиб во второй битве пр...,190,0.189,летом 1450 года пойнингс присоединился к восс...,74,0.823,был схвачен но выдержал пытки и смог бежать
3,97,0.161,историки полагают что джерард не был знаком к...,200,0.177,восста́ние дже́ка кэ́да — народное восстание ...,185,0.822,заседал в парламенте погиб во второй битве пр...
4,190,0.146,летом 1450 года пойнингс присоединился к восс...,185,0.158,заседал в парламенте погиб во второй битве пр...,137,0.820,однако став королевой она расправилась с теми...
...,...,...,...,...,...,...,...,...,...
216,136,0.000,елизавета единокровная сестра марии провела в...,136,0.000,елизавета единокровная сестра марии провела в...,111,0.725,одним из самых больших в 1078 году стал тауэр
217,135,0.000,теперь настало время протестантам сложить голову,135,0.000,теперь настало время протестантам сложить голову,175,0.724,смотрители королевской сокровищницы охраняют ...
218,134,0.000,не теряя времени новая королева приказала обе...,134,0.000,не теряя времени новая королева приказала обе...,104,0.723,в 1609 году джерард написал для иезуитского р...
219,133,0.000,когда через шесть лет эдуард умер английская ...,133,0.000,когда через шесть лет эдуард умер английская ...,153,0.721,в начале xiii века иоанн безземельный содержа...


In [ ]:
game_rel = get_relevance_table('Путь к новым союзникам (на илл.) в компьютерной игре укажет моральный компас.')

In [ ]:
from sklearn.metrics import ndcg_score

In [ ]:
def count_ndcg(relevances: pd.DataFrame, notable_sentences: Dict[int, int]):
  y_true = np.zeros((relevances.shape[0]))
  tfidf_score = relevances.relevance_tfidf.to_numpy()[relevances.sentence_id_t.to_numpy()]
  count_score = relevances.relevance_countv.to_numpy()[relevances.sentence_id_c.to_numpy()]
  torch_score = relevances.relevance_pytorch.to_numpy()[relevances.sentence_id_p.to_numpy()]
  for sentence_id, true_relevance in notable_sentences.items():
    y_true[sentence_id] = true_relevance
  print(
      [
          f'Count score: {ndcg_score([y_true], [count_score])}',
          f'TFidff score: {ndcg_score([y_true], [tfidf_score])}',
          f'Torch score: {ndcg_score([y_true], [torch_score])}'
      ], sep='\n'
  )

In [ ]:
count_ndcg(game_rel, relevant_sentences['game'])

['Count score: 0.20070657495901015', 'TFidff score: 0.20070657495901015', 'Torch score: 0.2265627810652808']


In [ ]:
count_ndcg(riot_rel, relevant_sentences['riot'])

['Count score: 0.19765803791669861', 'TFidff score: 0.19765803791669861', 'Torch score: 0.18628129181797048']


In [ ]:
count_ndcg(izu_rel, relevant_sentences['izuerit'])

['Count score: 0.30699571911820367', 'TFidff score: 0.3063520285016575', 'Torch score: 0.17087198123340605']
